In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import requests
import urllib.parse
import regex as re

In [ ]:
NAT_DIS_PATH = "transformed_emdat_natural_disasters.xlsx"

In [ ]:
df = pd.read_excel(NAT_DIS_PATH)
df.head()

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df[df["Start Year"] != df["End Year"]][["Year", "Start Year", "End Year"]]

In [ ]:
df.Seq.value_counts

In [ ]:
df = df.drop(["Seq", "Start Day", "Start Month", "Start Year", "End Day", "End Month", "End Year", "Local Time"], axis=1)
df

In [ ]:
df["Disaster Group"].unique()

In [ ]:
df["Disaster Subgroup"].unique()

In [ ]:
df = df[df["Disaster Subgroup"] != "Technological"]

In [ ]:
df["Disaster Subtype"].unique()

In [ ]:
subtypes_to_remove = ['Explosion', 'Ground movement', 'Ash fall', 'Water', 'Rail', 'Collapse', 'Other', 'Air', 'Lava flow', 'Volcanic activity',\
    'Animal accident', 'Pyroclastic flow ', 'Earthquake']
df = df[~df["Disaster Subtype"].isin(subtypes_to_remove)]
df

In [ ]:
'''
# Import the required library
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import random

def do_geocode(location):
    geolocator = Nominatim(user_agent=f"VA_Project{random.random()}")
    
    return geolocator.geocode(location)


# Initialize Nominatim API
locations = set(df["Location"].unique())

#df['der_coords'] = df['Location'].apply(geolocator.geocode)
#der_coords = [do_geocode(location) for location in locations]
#der_coords
locations
'''

In [72]:
def find_coords_for(row):
    location = row["Location"]
    country = row["Country"]
    return location_to_coords(location, country)

def location_to_coords(location, country):
    # if no location, use country
    if pd.isnull(location):
        return country_to_coords(country)

    # trying using location
    url = f'https://nominatim.openstreetmap.org/search/{urllib.parse.quote(location)}?format=json'
    response = requests.get(url).json()
    
    # trying using the last part of location
    if not response:
        new_location = re.split(r'[,!.:]+', location)[-1].strip()
        if new_location != location:
            return location_to_coords(new_location, country)
        # using the country if also this attempt failed
        return country_to_coords(country)
    
    lat, lon = response[0]['lat'], response[0]['lon']
    return (lat, lon)

def country_to_coords(country):
    #TODO clean this code
    if "Germany" in country:
        country = "Germany"
    elif "Taiwan" in country:
        country = "Taiwan"
    elif "Yemen" in country:
        country = "Yemen"
    elif "Macedonia (the former Yugoslav Republic of)" in country:
        country = "North Macedonia"
    elif country=="Sint Maarten (Dutch part)":
        country = "Sint Maarten"
        
    url = f'https://nominatim.openstreetmap.org/search/{urllib.parse.quote(country)}?format=json'
    response = requests.get(url).json()
    if not response:
        print(f"ERROR EVEN IN COUNTRY: {country}")
        return (0, 0) #TODO fix rows with 0,0
    lat, lon = response[0]['lat'], response[0]['lon']
    return (lat, lon)



In [ ]:
for i, row in df.iterrows():
    
    if pd.isnull(row["Latitude"]) or pd.isnull(row["Longitude"]):
        lat, lon = find_coords_for(row)
        df.at[i, 'Latitude'] = lat
        df.at[i, 'Longitude'] = lon
    else:
        continue
df

In [ ]:
df["Latitude"].isna().sum()

In [ ]:
df.to_csv("EMDAT_disasters.csv", index=True, index_label="index")

In [67]:
df = pd.read_csv("EMDAT_disasters.csv")
df

,index,Year,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Country,Region,Continent,Location,Dis Mag Scale,Latitude,Longitude,Total Deaths,No Injured,No Affected,No Homeless,Total Affected
0,0,1900,Natural,Climatological,Drought,Drought,Drought,Cabo Verde,Western Africa,Africa,Countrywide,Km2,52.088299199999994,-2.157944832014875,11000.0,NaN,NaN,NaN,NaN
1,1,1900,Natural,Climatological,Drought,Drought,Drought,India,Southern Asia,Asia,Bengal,Km2,39.4735045,-85.9228997,1250000.0,NaN,NaN,NaN,NaN
2,6,1903,Natural,Geophysical,Mass movement (dry),Rockfall,Mass movement (dry) / Rockfall,Canada,Northern America,Americas,"Frank, Alberta",NaN,49.6016471,-114.406023,76.0,23.0,NaN,NaN,23.0
3,10,1904,Natural,Meteorological,Storm,Tropical cyclone,Storm / Tropical cyclone,Bangladesh,Southern Asia,Asia,Chittagong,Kph,22.3336887,91.8342162,NaN,NaN,NaN,NaN,NaN
4,12,1905,Natural,Geophysical,Mass movement (dry),Rockfall,Mass movement (dry) / Rockfall,Canada,Northern America,Americas,"Spence's Bridge, British Columbia",NaN,55.001251,-125.002441,18.0,18.0,NaN,NaN,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14636,25641,2022,Natural,Meteorological,Storm,Convective storm,Storm / Convective storm,Congo (the Democratic Republic of the),Middle Africa,Africa,NaN,Kph,-2.9814344,23.8222636,26.0,NaN,NaN,NaN,NaN
14637,25642,2022,Natural,Biological,Epidemic,Viral disease,Epidemic / Viral disease,Congo (the Democratic Republic of the),Middle Africa,Africa,"Rethy, Angumu (Ituri)",Vaccinated,2.0238234999999998,31.0470203672303,2.0,NaN,91738.0,NaN,91738.0
14638,25643,2022,Natural,Biological,Epidemic,Viral disease,Epidemic / Viral disease,Congo (the Democratic Republic of the),Middle Africa,Africa,"Wangata, Mbandaka (Equateur Province)",Vaccinated,-2.9814344,23.8222636,5.0,141.0,NaN,NaN,141.0
14639,25646,2022,Natural,Biological,Epidemic,Viral disease,Epidemic / Viral disease,Timor-Leste,South-Eastern Asia,Asia,"Aileu, Ainaro, Baucau, Bobonaro, Covalima, Dil...",Vaccinated,-8.7443169,126.063482,20.0,NaN,1353.0,NaN,1353.0


In [73]:

for i, row in df.iterrows():
    if row["Latitude"] == 0 or row["Longitude"]== 0:
        lat, lon = find_coords_for(row)
        df.at[i, 'Latitude'] = lat
        df.at[i, 'Longitude'] = lon
        print(i)

print(i)
df[(df["Latitude"] == 0) | (df["Longitude"]== 0)]

13263
14640


,index,Year,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Country,Region,Continent,Location,Dis Mag Scale,Latitude,Longitude,Total Deaths,No Injured,No Affected,No Homeless,Total Affected


In [74]:
df.drop("index", axis=1, inplace=True)
df.to_csv("EMDAT_disasters.csv", index=True, index_label="index")

In [77]:
df = pd.read_csv("EMDAT_disasters.csv")

for i, row in df.iterrows():
    try:
        float(row["Latitude"])
        float(row["Longitude"])
    except:
        print(row["Latitude"], row["Longitude"])
        df.at[i, 'Latitude'] = float(row["Latitude"][:-2])
        df.at[i, 'Longitude'] = float(row["Longitude"][:-2])
df.to_csv("EMDAT_disasters.csv")

In [7]:
df = pd.read_csv("EMDAT_disasters.csv")
to_remove = ["Complex Disasters", "Extra-terrestrial", "Geophysical"]
df = df[~df["Disaster Subgroup"].isin(to_remove)]
df.drop(["Unnamed: 0"], axis=1, inplace=True)
df.to_csv("EMDAT_disasters.csv", index=False)

In [13]:
df = pd.read_csv("EMDAT_disasters.csv", index_col="index")
df.drop(["Disaster Group", "Continent", "Dis Mag Scale", "Total Deaths", "No Injured", "No Affected", "No Homeless", "Total Affected"], axis=1, inplace=True)
df.to_csv("EMDAT_disasters_reduced.csv", index=True)

In [3]:
df = pd.read_csv("EMDAT_disasters_reduced.csv", index_col="index")
df.replace("Extreme temperature ", "Extreme temperature", inplace=True)
df.to_csv("EMDAT_disasters_reduced_.csv", index=True)

In [47]:
df = pd.read_csv("EMDAT_disasters.csv")
df = df[df["Year"].isin([1970, 1995, 2019])]
df = df[~df["Disaster Subgroup"].isin(["Geophysical", "Complex Disasters" ])] #TODO decide for "Biological"
df = df.groupby(["Year", "Disaster Subgroup"])["index"].count().reset_index().rename(columns={"index": "frequency"})
df = df.pivot(index="Disaster Subgroup", columns='Year')
df.columns = df.columns.droplevel(0)
df.to_csv("EMDAT_disasters_barchart.csv")
df

Year,1970,1995,2019
Disaster Subgroup,,,
Biological,4,29,45
Climatological,3,13,29
Hydrological,37,109,220
Meteorological,24,93,112
